In [1]:
%pylab inline
import pandas as pd
import os

# NLP
import nltk

# Seaborn
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


ModuleNotFoundError: No module named 'seaborn'

In [50]:
# Read file into pandas
# 3 files are yelp_labelled.txt, amazon_cells_labelled.txt, imdb_labelled.txt
txt_file = 'amazon_cells_labelled.txt';
df = pd.read_table(txt_file, sep='\t')
df.columns = ['reviews','sentiment']
df['len'] = [len(x) for x in df['reviews']]

In [5]:
pos = df[(df['sentiment'] == 1)]
neg = df[(df['sentiment'] == 0)]
print(pos)

                                               reviews  sentiment  len
0                          Good case, Excellent value.          1   27
1                               Great for the jawbone.          1   22
3                                    The mic is great.          1   17
6          If you are Razr owner...you must have this!          1   43
9                      And the sound quality is great.          1   31
10   He was very impressed when going from the orig...          1   83
12                            Very good quality though          1   24
14   Highly recommend for any one who has a blue to...          1   56
16                                    So Far So Good!.          1   16
17                                       Works great!.          1   13
20   I bought this to use with my Kindle Fire and a...          1   65
22   I have yet to run this new battery below two b...          1   89
24                Great Pocket PC / phone combination.          1   36
25   I

In [6]:
def count_unique(words):
    uniq = set()
    for sentence in words:
        for word in sentence:
            uniq.add(word)
    return len(uniq)

In [51]:
#Remove problems with encoding
df['reviews'] = df['reviews'].apply(lambda x: unicode(x, errors="ignore"))

TypeError: decoding str is not supported

In [52]:
#tokenize without the punctuation
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
# make reviews lowercase
df['reviews'] = df['reviews'].apply(lambda x: x.lower())
# tokenize
df['reviews'] = df['reviews'].apply(lambda x: tokenizer.tokenize(x))
# visual check
df.head()

,reviews,sentiment,len
0,"[good, case, excellent, value]",1,27
1,"[great, for, the, jawbone]",1,22
2,"[tied, to, charger, for, conversations, lastin...",0,79
3,"[the, mic, is, great]",1,17
4,"[i, have, to, jiggle, the, plug, to, get, it, ...",0,74


In [9]:
# Check feature count
count_unique(df['reviews'])

1864

In [10]:
train= df[:int(df.shape[0]*0.7)]
test = df[int(df.shape[0]*0.7):]

print(test)

                                               reviews  sentiment  len
699  [also, the, phone, doesn, t, seem, to, accept,...          0  107
700         [you, can, t, beat, the, price, on, these]          1   34
701  [comes, with, a, strong, light, that, you, can...          1  114
702                    [lightweight, and, works, well]          1   27
703                                 [so, just, beware]          0   15
704  [i, love, this, cable, it, allows, me, to, con...          1   73
705  [so, anyone, near, you, will, hear, part, of, ...          0   55
706  [this, is, cool, because, most, cases, are, ju...          1   96
707  [i, bought, this, phone, as, a, replacement, f...          0   80
708  [appears, to, actually, outperform, the, origi...          1   87
709  [can, t, store, anything, but, phone, numbers,...          0   46
710  [it, lasts, less, than, 3o, minutes, if, i, ac...          0  115
711                                    [poor, quality]          0   13
712   

In [61]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

from sklearn.linear_model import LogisticRegression
from nltk.classify.scikitlearn import SklearnClassifier
logreg = SklearnClassifier(LogisticRegression())

In [62]:
#POS Tagging/Removal
# POS tag
df['posr']= df['reviews'].apply(lambda x: nltk.pos_tag(x))
# Leave the non POS removal ones alone
df['nposr'] = df['reviews']



In [63]:
# Collection of adjectives, nouns, adverbs and verbs to keep
pos_keep = ["JJ","JJR","JJS","NN","NNP","NNS","RB","RBR","VB","VBD","VBG","VBN","VBZ"]


def remove_pos(full):
    redc =[]
    for pair in full:
        if pair[1] in pos_keep:
            redc.append(pair[0])
    return redc
# Remove the words that are not in pos_keep
df['posr']= df['posr'].apply(lambda x: remove_pos(x))

df.head()

,reviews,sentiment,len,posr,nposr
0,"[good, case, excellent, value]",1,27,"[good, case, excellent, value]","[good, case, excellent, value]"
1,"[great, for, the, jawbone]",1,22,"[great, jawbone]","[great, for, the, jawbone]"
2,"[tied, to, charger, for, conversations, lastin...",0,79,"[tied, charger, conversations, lasting, more, ...","[tied, to, charger, for, conversations, lastin..."
3,"[the, mic, is, great]",1,17,"[mic, is, great]","[the, mic, is, great]"
4,"[i, have, to, jiggle, the, plug, to, get, it, ...",0,74,"[i, jiggle, plug, get, line, up, right, get, d...","[i, have, to, jiggle, the, plug, to, get, it, ..."


In [16]:
print(count_unique(df['posr']))

1691


In [64]:
#remove stopwords
# Import the list of stopwords from NLTK
from nltk.corpus import stopwords

# Remove the stopwords and store in "No Stops" column
stop = set(stopwords.words('english'))
# Ensure "not" is kept
stop.remove("not")
df['posr_stpd'] = df['posr'].apply(lambda x: [item for item in x if item not in stop])
df['nposr_stpd'] = df['nposr'].apply(lambda x: [item for item in x if item not in stop])
df['posr_nstpd'] = df['posr']
df['nposr_nstpd'] = df['nposr']
# visual check
df.head()

,reviews,sentiment,len,posr,nposr,posr_stpd,nposr_stpd,posr_nstpd,nposr_nstpd
0,"[good, case, excellent, value]",1,27,"[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]"
1,"[great, for, the, jawbone]",1,22,"[great, jawbone]","[great, for, the, jawbone]","[great, jawbone]","[great, jawbone]","[great, jawbone]","[great, for, the, jawbone]"
2,"[tied, to, charger, for, conversations, lastin...",0,79,"[tied, charger, conversations, lasting, more, ...","[tied, to, charger, for, conversations, lastin...","[tied, charger, conversations, lasting, minute...","[tied, charger, conversations, lasting, 45, mi...","[tied, charger, conversations, lasting, more, ...","[tied, to, charger, for, conversations, lastin..."
3,"[the, mic, is, great]",1,17,"[mic, is, great]","[the, mic, is, great]","[mic, great]","[mic, great]","[mic, is, great]","[the, mic, is, great]"
4,"[i, have, to, jiggle, the, plug, to, get, it, ...",0,74,"[i, jiggle, plug, get, line, up, right, get, d...","[i, have, to, jiggle, the, plug, to, get, it, ...","[jiggle, plug, get, line, right, get, decent, ...","[jiggle, plug, get, line, right, get, decent, ...","[i, jiggle, plug, get, line, up, right, get, d...","[i, have, to, jiggle, the, plug, to, get, it, ..."


In [65]:
# check feature count
count_unique(df['posr_stpd'])

1636

In [66]:
#stemming
ps = nltk.PorterStemmer()
ss = nltk.SnowballStemmer('english')
ls = nltk.LancasterStemmer()

# No Stemming
df['nposr_nstpd_nstem'] = df['nposr_nstpd']
df['posr_nstpd_nstem'] = df['posr_nstpd']
df['nposr_stpd_nstem'] = df['nposr_stpd']
df['posr_stpd_nstem'] = df['posr_stpd']
# Porter
df['nposr_nstpd_port'] = df['nposr_nstpd'].apply(lambda x: [ps.stem(y) for y in x])
df['posr_nstpd_port'] = df['posr_nstpd'].apply(lambda x: [ps.stem(y) for y in x])
df['nposr_stpd_port'] = df['nposr_stpd'].apply(lambda x: [ps.stem(y) for y in x])
df['posr_stpd_port'] = df['posr_stpd'].apply(lambda x: [ps.stem(y) for y in x])
df.head()

,reviews,sentiment,len,posr,nposr,posr_stpd,nposr_stpd,posr_nstpd,nposr_nstpd,nposr_nstpd_nstem,posr_nstpd_nstem,nposr_stpd_nstem,posr_stpd_nstem,nposr_nstpd_port,posr_nstpd_port,nposr_stpd_port,posr_stpd_port
0,"[good, case, excellent, value]",1,27,"[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excellent, value]","[good, case, excel, valu]","[good, case, excel, valu]","[good, case, excel, valu]","[good, case, excel, valu]"
1,"[great, for, the, jawbone]",1,22,"[great, jawbone]","[great, for, the, jawbone]","[great, jawbone]","[great, jawbone]","[great, jawbone]","[great, for, the, jawbone]","[great, for, the, jawbone]","[great, jawbone]","[great, jawbone]","[great, jawbone]","[great, for, the, jawbon]","[great, jawbon]","[great, jawbon]","[great, jawbon]"
2,"[tied, to, charger, for, conversations, lastin...",0,79,"[tied, charger, conversations, lasting, more, ...","[tied, to, charger, for, conversations, lastin...","[tied, charger, conversations, lasting, minute...","[tied, charger, conversations, lasting, 45, mi...","[tied, charger, conversations, lasting, more, ...","[tied, to, charger, for, conversations, lastin...","[tied, to, charger, for, conversations, lastin...","[tied, charger, conversations, lasting, more, ...","[tied, charger, conversations, lasting, 45, mi...","[tied, charger, conversations, lasting, minute...","[tie, to, charger, for, convers, last, more, t...","[tie, charger, convers, last, more, minut, maj...","[tie, charger, convers, last, 45, minut, major...","[tie, charger, convers, last, minut, major, pr..."
3,"[the, mic, is, great]",1,17,"[mic, is, great]","[the, mic, is, great]","[mic, great]","[mic, great]","[mic, is, great]","[the, mic, is, great]","[the, mic, is, great]","[mic, is, great]","[mic, great]","[mic, great]","[the, mic, is, great]","[mic, is, great]","[mic, great]","[mic, great]"
4,"[i, have, to, jiggle, the, plug, to, get, it, ...",0,74,"[i, jiggle, plug, get, line, up, right, get, d...","[i, have, to, jiggle, the, plug, to, get, it, ...","[jiggle, plug, get, line, right, get, decent, ...","[jiggle, plug, get, line, right, get, decent, ...","[i, jiggle, plug, get, line, up, right, get, d...","[i, have, to, jiggle, the, plug, to, get, it, ...","[i, have, to, jiggle, the, plug, to, get, it, ...","[i, jiggle, plug, get, line, up, right, get, d...","[jiggle, plug, get, line, right, get, decent, ...","[jiggle, plug, get, line, right, get, decent, ...","[i, have, to, jiggl, the, plug, to, get, it, t...","[i, jiggl, plug, get, line, up, right, get, de...","[jiggl, plug, get, line, right, get, decent, v...","[jiggl, plug, get, line, right, get, decent, v..."


In [67]:
#test and evaluate with NB and LR
#Function to Convert the Data into a Feature Set
## Transform data into list of ([tokens],sentiment label)
def createTrainingDataNLTK(sentences,labels):
    rdata = np.vstack([sentences,labels])
    rdata = np.transpose(rdata)
    data = list();
    for i in range(0,len(rdata)):
        tokens = rdata[i][0].split(" ")
        d_tuple = (tokens, rdata[i][1]);
        data.append(d_tuple)
    return data;


In [68]:
#CREATE TRAINING DATA
# create a list of all possible feature reduction permutations in the order of stopword removal_pos_stem
featurereductions = ['nposr_nstpd_nstem', 'posr_nstpd_nstem', 
                     'nposr_stpd_nstem', 'posr_stpd_nstem', 
                     'nposr_nstpd_port', 'nposr_nstpd_snow', 'nposr_nstpd_lanc', 'nposr_stpd_port', 'nposr_stpd_snow', 'nposr_stpd_lanc',
                     'posr_nstpd_port', 'posr_nstpd_snow', 'posr_nstpd_lanc', 'posr_stpd_port', 'posr_stpd_snow', 'posr_stpd_lanc']

In [69]:
# merge the words into sentence to use current implementation of createTrainingData
def create_nltk_train_data (feature_reduction):
    df['sentences'] = df[feature_reduction].apply(lambda x: " ".join(x).encode('UTF-8'))
    x_label = "sentences"
    y_label = "sentiment"
    nltk_train_data = createTrainingDataNLTK(df[x_label],df[y_label])
    return nltk_train_data

In [70]:
# returns the accuracy of the test data for Naive Bayes
# when predicted against fitted training data
def train1(training_set,test_set):
    
    sentim_analyzer = SentimentAnalyzer()
    all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_set])
    unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
    sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
    
    training_set = sentim_analyzer.apply_features(training_set)
    test_set = sentim_analyzer.apply_features(test_set)
                                              
    trainer = NaiveBayesClassifier.train
    classifier = sentim_analyzer.train(trainer, training_set)
    results = sentim_analyzer.evaluate(test_set).items()                             
    return results

In [71]:
# returns the accuracy of the test data for Logistic Regression
# when predicted against fitted training data
def train2(training_set,test_set):
    
    sentim_analyzer = SentimentAnalyzer()
    all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_set])
    unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
    sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
    
    training_set = sentim_analyzer.apply_features(training_set)
    test_set = sentim_analyzer.apply_features(test_set)
                                              
    trainer = logreg.train
    classifier = sentim_analyzer.train(trainer, training_set)
    results = sentim_analyzer.evaluate(test_set).items()                             
    return results

In [78]:
## A custom parser function to interpret results of training function
def nltk_parser(results,summer):
    for key,value in sorted(results):
        #print('{0}: {1}'.format(key, value))
        if key == "Accuracy": summer += value;
    
    return summer

In [81]:
## k-fold cross validation designed to be agnostic to training data and training function
def k_fold_validation(k,data,train_func,parser):
    
    b_length = len(data)/k; # the testing bucket length
    sum_acc = 0; # Use this for average accuracy
    print (k, "- fold cross validation")
    print ("-----------------------------")
    for i in range(0,k):       
        s_i = i*b_length; # testing interval start index
        f_i = s_i + b_length; # testing interval end index
        test_data = data[s_i:f_i]
        train_data = data[0:s_i]+data[f_i:len(data)];
        metrics = train_func(train_data,test_data);
        sum_acc = parser(metrics,sum_acc);

    print ('\nAverage Accuracy:', sum_acc / k);

In [79]:
## k-fold cross validation designed to be agnostic to training data and training function
# Returns average accuracy
def k_fold_validation_simple(k,data,train_func,parser):
    b_length = len(data)/k; # the testing bucket length
    sum_acc = 0; # Use this for average accuracy
    for i in range(0,k):       
        s_i = i*b_length; # testing interval start index
        f_i = s_i + b_length; # testing interval end index
        test_data = data[s_i:f_i]
        train_data = data[0:s_i]+data[f_i:len(data)];
        metrics = train_func(train_data,test_data);
        sum_acc = parser(metrics,sum_acc);
    return sum_acc / k;

In [86]:
#Naive Bayes testing
K = 5; # Constant for K-fold cross validation
accuracies =[]
dict_metrics_nb = {}
for i in range(0, len(featurereductions)):
    nltk_train_data = create_nltk_train_data (featurereductions[i])
    print (featurereductions[i])
    accuracies.append(k_fold_validation_simple(K,nltk_train_data,train1,nltk_parser))
    # creates a dict with featurereduction techniques, accuracy, and feature counts
    dict_metrics_nb.update({featurereductions[i]: accuracies[i]})
    print (accuracies[i])

TypeError: a bytes-like object is required, not 'str'

In [76]:
#LR 
K = 5; # Constant for K-fold cross validation
accuracies2 = []
dict_metrics_lr = {}
for i in range(0, len(featurereductions)):
    nltk_train_data = create_nltk_train_data (featurereductions[i])
    print (featurereductions[i])
    accuracies2.append(k_fold_validation_simple(K,nltk_train_data,train2,nltk_parser))
    # creates a dict with featurereduction techniques, accuracy, and feature counts
    dict_metrics_lr.update({featurereductions[i]: accuracies2[i]})
    print (accuracies2[i])

TypeError: a bytes-like object is required, not 'str'